<a href="https://colab.research.google.com/github/Antony-gitau/2024_ICME_Challenge/blob/main/config_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/LuoXishuang0712/VHR-BirdPose.git

Cloning into 'VHR-BirdPose'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 38 (delta 11), reused 35 (delta 11), pack-reused 0
Receiving objects: 100% (38/38), 28.75 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (11/11), done.


# preparing the environment according to HRNET

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!git clone https://github.com/leoxiaobin/deep-high-resolution-net.pytorch.git

Cloning into 'deep-high-resolution-net.pytorch'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 296 (delta 0), reused 2 (delta 0), pack-reused 293
Receiving objects: 100% (296/296), 15.56 MiB | 21.21 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [ ]:
!pip install -r /content/deep-high-resolution-net.pytorch/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for shapely: filename=Shapely-1.7.1-cp310-cp310-linux_x86_64.whl size=997441 sha256=f512e83ec17d506c42ef7244f82d0ad8c94d2c611c527c5e3531c6daca91ab99
  Stored in directory: /root/.cache/pip/wheels/2e/fa/97/c85f587c35afcaf4a81c481741d36592518d1e50445572f0d4
Successfully built shapely
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.6.4
    Uninstalling Shapely-1.6.4:
      Successfully uninstalled Shapely-1.6.4


###### changed the version of opencv and shapely

In [ ]:
!git clone https://github.com/sutdcv/Animal-Kingdom.git

Cloning into 'Animal-Kingdom'...
remote: Enumerating objects: 786, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 786 (delta 63), reused 114 (delta 49), pack-reused 646
Receiving objects: 100% (786/786), 169.05 MiB | 32.82 MiB/s, done.
Resolving deltas: 100% (224/224), done.
Updating files: 100% (433/433), done.


# Loading the animal kingdom data to dataloader

In [1]:
from torchvision.datasets import CocoDetection
import torchvision.transforms as transforms

In [2]:
from google.colab import drive
import tarfile

# Change directory to the dataset folder
%cd /content/drive/MyDrive/pose_estimation

# Specify the path to the tar.gz file
tar_file_path = '/content/drive/MyDrive/pose_estimation_dataset/dataset.tar.gz'

# Open the tar.gz file
with tarfile.open(tar_file_path, 'r') as tar:
  # Extract the contents of the tar.gz file
  tar.extractall(path='/content/dataset')

# Print a success message
print('Successfully extracted the tar.gz file')


[Errno 2] No such file or directory: '/content/drive/MyDrive/pose_estimation'
/content
Successfully extracted the tar.gz file


# loading the data into a dataloader in preparation for training

In [3]:
pip install torch torchvision


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import json
import os

class CustomAnimalDataset(Dataset):
    def __init__(self, dataset_folder, annotations_file, transform=None):
        """
        Args:
            dataset_folder (string): Path to the folder containing images.
            annotations_file (string): Path to the JSON file containing annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.dataset_folder = dataset_folder
        self.transform = transform
        with open(annotations_file) as f:
            self.annotations = json.load(f)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Extract information for the current image
        annotation = self.annotations[idx]
        img_path = os.path.join(self.dataset_folder, annotation['image'])
        image = Image.open(img_path).convert('RGB')

        # Handle transformations
        if self.transform:
            image = self.transform(image)

        # Extract other information from the annotation file
        joints_vis = annotation.get('joints_vis', [])
        scale = annotation.get('scale', 1.0)
        center = annotation.get('center', [0, 0])

        # Convert the lists in the annotation file to PyTorch tensors as required
        scale = torch.tensor(annotation['scale'], dtype=torch.float32)
        center = torch.tensor(annotation['center'], dtype=torch.float32)
        joints = torch.tensor(annotation.get('joints', []), dtype=torch.float32)
        joints_vis = torch.tensor(annotation.get('joints_vis', []), dtype=torch.float32)

        return image, {'scale': scale, 'center': center, 'joints': joints, 'joints_vis': joints_vis}

# Define your transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images as needed
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])



In [5]:
def create_dataloader(dataset_folder, annotations_file, batch_size=4, shuffle=True, num_workers=0):
    """
    Create a DataLoader for the custom animal dataset.

    Args:
        dataset_folder (string): Path to the folder containing images.
        annotations_file (string): Path to the JSON file containing annotations.
        batch_size (int): How many samples per batch to load.
        shuffle (bool): Set to True to have the data reshuffled at every epoch.
        num_workers (int): How many subprocesses to use for data loading.

    Returns:
        DataLoader: A DataLoader for the custom dataset.
    """
    dataset = CustomAnimalDataset(
        dataset_folder=dataset_folder,
        annotations_file=annotations_file,
        transform=transform
    )

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    return dataloader

# usage with the dataset of provided animal kingdom /content/dataset
dataset_folder = '/content/dataset/dataset'
annotations_file = '/content/drive/MyDrive/pose_estimation_dataset/annotation/ak_P3_bird/train.json'


dataloader = create_dataloader(dataset_folder, annotations_file, batch_size=4, shuffle=True, num_workers=0)

In [6]:
def print_dataloader(dataloader):
  for i, batch in enumerate(dataloader):
    if i >= 1: # can change to see more than 1 batch
      break
    print(len(batch))
    print(batch)

print_dataloader(dataloader)

2
[tensor([[[[0.3647, 0.3490, 0.3255,  ..., 0.0510, 0.0667, 0.0745],
          [0.3569, 0.3451, 0.3216,  ..., 0.0510, 0.0627, 0.0745],
          [0.3490, 0.3373, 0.3176,  ..., 0.0549, 0.0667, 0.0784],
          ...,
          [0.5098, 0.5882, 0.5882,  ..., 0.1961, 0.1922, 0.1922],
          [0.3412, 0.5686, 0.5882,  ..., 0.1961, 0.1922, 0.1922],
          [0.2196, 0.5216, 0.5882,  ..., 0.1961, 0.1961, 0.1961]],

         [[0.4627, 0.4471, 0.4235,  ..., 0.0824, 0.0902, 0.0863],
          [0.4549, 0.4431, 0.4157,  ..., 0.0824, 0.0863, 0.0863],
          [0.4471, 0.4353, 0.4157,  ..., 0.0824, 0.0902, 0.0902],
          ...,
          [0.5529, 0.6275, 0.6314,  ..., 0.2118, 0.2078, 0.2078],
          [0.3765, 0.6039, 0.6314,  ..., 0.2118, 0.2078, 0.2078],
          [0.2549, 0.5529, 0.6275,  ..., 0.2118, 0.2118, 0.2118]],

         [[0.1451, 0.1294, 0.1137,  ..., 0.0000, 0.0039, 0.0000],
          [0.1373, 0.1255, 0.1059,  ..., 0.0000, 0.0000, 0.0000],
          [0.1294, 0.1176, 0.1020,  ...

Now that we have a training data loader that has the annotations we care about and the image, now we can try to come up with a baseline model.

this includes:  defining the model architecture, setting up the loss function, and running the training loop.

this phd student has implemented deeplabv3 -> https://github.com/chenxi116/DeepLabv3.pytorch
there are a bunch of backbone models we could use from here -> https://github.com/open-mmlab/mmpose/blob/main/mmpose/models/backbones/__init__.py
